<a href="https://colab.research.google.com/github/kimseoa/test/blob/master/naver%20%EB%A6%AC%EB%B7%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#가장 도움되었던 참고사이트
#https://medium.com/@adam.wearne/lets-get-sentimental-with-pytorch-dcdd9e1ea4c9
#https://github.com/pytorch/text/blob/master/torchtext/data/field.py
#https://simonjisu.github.io/nlp/2018/07/18/torchtext.html

In [0]:
from torchtext.data import TabularDataset
from torchtext.data import Field
from torchtext.data import LabelField
import torch

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from torchtext import data
TEXT = data.Field(tokenize = 'spacy',include_lengths = True)

# TEXT = Field(sequential=True,#데이터 유형이 순차 데이터를 나타내는 지 여부. 거짓이면토큰 화가 적용되지 않습니다. 기본값 : True
#              use_vocab=True, #Vocab 객체 사용 여부. False이면이 데이터 필드는 이미 숫자 여야합니다. 기본값 : True
#              tokenize=str.split, #문자열을 토큰 화하는 데 사용되는 함수
#              lower=True, #텍스트를 소문자로
#              batch_first=True)   #배치 치수가 먼저 텐서를 생성할지 여부.기본값 : False
LABEL = LabelField(dtype=torch.float) #is designed to hold labels!

In [0]:
train_data = TabularDataset(path='/content/gdrive/My Drive/mountfile/ratings_train2.txt', format='tsv', fields=[('id', None),('text', TEXT), ('label', LABEL)],skip_header = True,filter_pred = lambda x: len(x.text) > 1)
test_data = TabularDataset(path='/content/gdrive/My Drive/mountfile/ratings_test2.txt', format='tsv', fields=[('id', None),('text', TEXT), ('label', LABEL)],skip_header = True, filter_pred = lambda x: len(x.text) > 1 )

In [0]:
import random

In [0]:

#Field객체를 인스턴스화하고 데이터를로드

train_data, valid_data = train_data.split(random_state = random.seed(0)) #Field객체를 인스턴스화하고 데이터를로드

In [0]:
print(vars(train_data[0]))

{'text': ['키스한방에', '인생이', '역전되는', '한국드라마식', '영화', '하지만', '내용은최고엿다', '그치만', '평점이', '너무나도', '사기적이기에', '너무', '질투나서', '1점을', '준다', '실제론', '10점준거임'], 'label': '0'}


In [0]:

print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 98911
Number of validation examples: 42390
Number of testing examples: 47180


In [0]:
MAX_VOCAB_SIZE = 50000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE,
                 vectors = "glove.6B.100d",   #임베딩 에는 나타나지 않는 단어 는 모두 0으로 구성된 벡터로 초기화
                 unk_init = torch.Tensor.normal_) # 정규 분포에서 선택한 항목을 가진 벡터로 해당 단어를 설정(속도 빠르게)

LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [00:43, 19.7MB/s]                           
100%|█████████▉| 398331/400000 [00:20<00:00, 19712.94it/s]

In [0]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 50002
Unique tokens in LABEL vocabulary: 2


In [0]:

print(TEXT.vocab.freqs.most_common(20))

[('..', 21258), ('!', 19103), ('...', 16583), (',', 11631), ('?', 10626), ('영화', 10137), ('너무', 5928), ('정말', 5811), (';', 4680), ('진짜', 4484), ('....', 3822), ('이', 3620), ('영화.', 2534), ('그냥', 2423), ('왜', 2395), ('이런', 2349), ('더', 2293), ('"', 2230), ('수', 2085), ('영화를', 1931)]


In [0]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f1c3bd56c80>, {'0': 0, '1': 1})


In [0]:
from torchtext import data
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device,sort_within_batch = True,sort_key = lambda x: len(x.text)) # sort_within_batch:(self.sort_key에 따라 내림차순 한다 꼭 써야함. sort_key:비슷한 길이의 예제를 함께 배치하고 패딩을 최소화하기 위해 예제를 정렬하는 데 사용하는 키(꼭 써야함)


In [0]:
import torch.nn as nn
import torch.nn.functional as F

import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(text))
        
        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        #hidden = [batch size, hid dim * num directions]
            
        return self.fc(hidden)

In [0]:

INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

In [0]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 7,310,857 trainable parameters


In [0]:
pretrained_embeddings = TEXT.vocab.vectors  #vectors = "glove.6B.100d", 숫자 변경하면 torch size 변경됨

print(pretrained_embeddings.shape)

torch.Size([50002, 100])


In [0]:

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 2.9700e-01, -3.7852e-01, -1.8263e+00,  ...,  6.9256e-01,
          6.1497e-01,  6.9305e-01],
        [ 1.0096e+00, -6.7647e-01, -3.9467e-01,  ...,  9.0620e-01,
         -1.0057e+00,  5.2581e-01],
        [-1.9256e+00, -1.3820e+00, -3.0119e-02,  ...,  6.6690e-01,
          2.0571e+00,  7.6489e-01],
        ...,
        [-8.3850e-01,  1.9119e+00,  1.3825e-03,  ..., -1.4235e+00,
         -1.0343e+00,  3.1351e-01],
        [-1.0747e+00,  3.8127e-01, -5.7731e-01,  ..., -5.4906e-01,
         -3.4272e-01,  1.1652e+00],
        [ 6.8391e-01, -5.5642e-02,  7.6247e-01,  ...,  2.3206e-01,
          7.2023e-02, -9.5438e-01]])

In [0]:

UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [-1.9256e+00, -1.3820e+00, -3.0119e-02,  ...,  6.6690e-01,
          2.0571e+00,  7.6489e-01],
        ...,
        [-8.3850e-01,  1.9119e+00,  1.3825e-03,  ..., -1.4235e+00,
         -1.0343e+00,  3.1351e-01],
        [-1.0747e+00,  3.8127e-01, -5.7731e-01,  ..., -5.4906e-01,
         -3.4272e-01,  1.1652e+00],
        [ 6.8391e-01, -5.5642e-02,  7.6247e-01,  ...,  2.3206e-01,
          7.2023e-02, -9.5438e-01]])


In [0]:

import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [0]:

criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [0]:

def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [0]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:

import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [0]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

100%|█████████▉| 398331/400000 [00:40<00:00, 19712.94it/s]

Epoch: 01 | Epoch Time: 0m 33s
	Train Loss: 0.603 | Train Acc: 66.23%
	 Val. Loss: 0.510 |  Val. Acc: 75.15%
Epoch: 02 | Epoch Time: 0m 33s
	Train Loss: 0.501 | Train Acc: 74.83%
	 Val. Loss: 0.447 |  Val. Acc: 78.46%
Epoch: 03 | Epoch Time: 0m 32s
	Train Loss: 0.443 | Train Acc: 78.38%
	 Val. Loss: 0.419 |  Val. Acc: 80.10%
Epoch: 04 | Epoch Time: 0m 32s
	Train Loss: 0.405 | Train Acc: 80.77%
	 Val. Loss: 0.403 |  Val. Acc: 80.91%
Epoch: 05 | Epoch Time: 0m 32s
	Train Loss: 0.376 | Train Acc: 82.48%
	 Val. Loss: 0.401 |  Val. Acc: 81.32%
